# Description
This kernel performs inference for [PANDA concat tile pooling starter](https://www.kaggle.com/iafoss/panda-concat-fast-ai-starter) kernel with use of multiple models and 8 fold TTA. Check it for more training details. The image preprocessing pipline is provided [here](https://www.kaggle.com/iafoss/panda-16x128x128-tiles).

In [ ]:
import cv2
from tqdm import tqdm_notebook as tqdm
import fastai
from fastai.vision import *
import os
from mish_activation import *
import warnings
warnings.filterwarnings("ignore")
import skimage.io
import numpy as np
import pandas as pd
sys.path.insert(0, '../input/semisupervised-imagenet-models/semi-supervised-ImageNet1K-models-master/')
from hubconf import *

In [ ]:
os.listdir('../input/mypandago/')

In [ ]:
DATA = '../input/prostate-cancer-grade-assessment/test_images'
TEST = '../input/prostate-cancer-grade-assessment/test.csv'
SAMPLE = '../input/prostate-cancer-grade-assessment/sample_submission.csv'
MODELS = [f'../input/mypandago/RNXT50_200_reg_nozoomin_0.pth']

sz = 128*4
img_size = 200
bs = 2
N = 12
nworkers = 2

In [ ]:
df = pd.read_csv('../input/mypanda/train_5folds.csv')
df = df[df.fold == 0]
df

# Model

In [ ]:
def _resnext(url, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    #state_dict = load_state_dict_from_url(url, progress=progress)
    #model.load_state_dict(state_dict)
    return model

# 101 [3, 4, 23, 3], 50 [3, 4, 6, 3]
class Model(nn.Module):
    def __init__(self, arch='resnext50_32x4d', n=1, pre=True):
        super().__init__()
        #m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', arch)
        m = _resnext(semi_supervised_model_urls[arch], Bottleneck, [3, 4, 6, 3], False, 
                progress=False,groups=32,width_per_group=4)
        self.enc = nn.Sequential(*list(m.children())[:-2])       
        nc = list(m.children())[-1].in_features
        self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.Linear(2*nc,512),
                Mish(),nn.BatchNorm1d(512),nn.Dropout(0.5),nn.Linear(512,n))
        
    def forward(self, x):
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        x = self.enc(x)
        shape = x.shape
        x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*n,shape[3])
        x = self.head(x)
        return x

In [ ]:
models = []
for path in MODELS:
    state_dict = torch.load(path,map_location=torch.device('cpu'))
    model = Model()
    
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
#         name = k
        new_state_dict[name] = v
        
    model.load_state_dict(new_state_dict)
    model.float()
    model.eval()
    model.cuda()
    models.append(model)

del state_dict

In [ ]:
# models = []
# for path in MODELS:
#     state_dict = torch.load(path,map_location=torch.device('cpu'))
#     model = Model()
#     model.load_state_dict(state_dict)
#     model.float()
#     model.eval()
#     model.cuda()
#     models.append(model)

# del state_dict

# Data

In [ ]:
def imshow(
    img,
    title=None,
    show_shape=True,
    figsize=(5, 5)
):
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(img)
    ax.grid("off")
    ax.set_xticks([])
    ax.set_yticks([])

    if show_shape:
        ax.set_xlabel(f"Shape: {img.shape}", fontsize=16)
        
    if title:
        ax.set_title(title, fontsize=16)

    return ax

In [ ]:
def crop_white(image: np.ndarray) -> np.ndarray:
    assert image.shape[2] == 3
    assert image.dtype == np.uint8
    ys, = (image.min((1, 2)) != 255).nonzero()
    xs, = (image.min(0).min(1) != 255).nonzero()
    if len(xs) == 0 or len(ys) == 0:
        return image
    return image[ys.min():ys.max() + 1, xs.min():xs.max() + 1]

In [ ]:
from PIL import Image

def tile(img):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    
#     img_ = np.ndarray([len(img),img_size,img_size,3])
    img_ = []
    for i in range(len(img)):
        img_tmp = img[i]
#         img_tmp = Image.fromarray(img_tmp)
#         img_tmp = img_tmp.resize((img_size,img_size))
#         img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_RGB2BGR)
        img_tmp = cv2.resize(img_tmp,(256,256))
        img_tmp = cv2.resize(img_tmp,(img_size,img_size))
        img_.append(img_tmp)
    img_ = np.stack(img_)
    return img_

mean = torch.tensor([1.0-0.90949707, 1.0-0.8188697, 1.0-0.87795304])
std = torch.tensor([0.36357649, 0.49984502, 0.40477625])

class PandaDataset(Dataset):
    def __init__(self, path, test, is_train=True):
        self.path = path
        if is_train:
            self.names = list(pd.read_csv(test).image_id)
        else:
            self.names = list(df.image_id)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img = skimage.io.MultiImage(os.path.join(DATA,name+'.tiff'))[-2]
#         img = crop_white(img)
        imgs = tile(img)
#         n = len(imgs)
#         imgs = np.resize(imgs, (n,img_size,img_size,3))
#         print(imgs.shape)
        tiles = torch.Tensor(1.0 - imgs/255.0)
        tiles = (tiles - mean)/std
        return tiles.permute(0,3,1,2), name

In [ ]:
img = skimage.io.MultiImage(os.path.join(DATA,'0005f7aaab2800f6170c399693a96917'+'.tiff'))[-2]
imgs = tile(img)
# img_tmp = cv2.cvtColor(imgs[0], cv2.COLOR_RGB2BGR)
# img_tmp = cv2.resize(imgs[0],(img_size,img_size))
# img_tmp = np.resize(imgs[0], (img_size,img_size,3))
imshow(imgs[0])

# Valid

In [ ]:
TRAINDATA = '../input/prostate-cancer-grade-assessment/train_images'
TRAIN = '../input/prostate-cancer-grade-assessment/train.csv'

In [ ]:
# if os.path.exists(TRAINDATA):
#     ds = PandaDataset(TRAINDATA,TRAIN,False)
#     dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
#     valid_names,valid_preds = [],[]

#     with torch.no_grad():
#         for x,y in tqdm(dl):
#             x = x.cuda()
#             #dihedral TTA
#             x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
#               x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
#               x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],1)
#             x = x.view(-1,N,3,img_size,img_size)
#             p = [model(x) for model in models]
#             p = torch.stack(p,1)
#             p = p.view(bs,8*len(models),-1).mean(1).cpu()
#             valid_names.append(y)
#             valid_preds.append(p)

In [ ]:
# valid_names = np.concatenate(valid_names)
# valid_preds = torch.cat(valid_preds).numpy()
# valid_labels = df.isup_grade.values

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json

In [ ]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5, 4.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
#         coef = [0.5, 1.5, 2.5, 3.5, 4.5]
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5
        return X_p
    
    def coefficients(self):
        return self.coef_['x']

In [ ]:
optR = OptimizedRounder()

In [ ]:
# optR.fit(valid_preds,valid_labels[:])
# coefficients = optR.coefficients()

In [ ]:
# coefficients

# Prediction

In [ ]:
sub_df = pd.read_csv(SAMPLE)
if os.path.exists(DATA):
    ds = PandaDataset(DATA,TEST)
    dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
    names,preds = [],[]

    with torch.no_grad():
        for x,y in tqdm(dl):
            x = x.cuda()
            #dihedral TTA
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
              x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
              x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],1)
            x = x.view(-1,N,3,img_size,img_size)
            p = [model(x) for model in models]
            p = torch.stack(p,1)
            p = p.view(bs,8*len(models),-1).mean(1).cpu()
            names.append(y)
            preds.append(p)
    
    names = np.concatenate(names)
    preds = torch.cat(preds).numpy()
#     sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})
#     sub_df.to_csv('submission.csv', index=False)
#     sub_df.head()

In [ ]:
coefficients = [0.538597, 1.585673, 2.633171, 3.240498, 4.070715]

In [ ]:
coefficients

In [ ]:
test_predictions = optR.predict(preds, coefficients)

In [ ]:
test_predictions = test_predictions.reshape(1,-1).tolist()

In [ ]:
names = names.reshape(1,-1).tolist()

In [ ]:
sub_df = pd.DataFrame({'image_id': names[0], 'isup_grade': test_predictions[0]})
sub_df["isup_grade"] = sub_df["isup_grade"].astype(int)

In [ ]:
sub_df.to_csv("submission.csv", index=False)
sub_df.head()